In [1]:
import numpy as np
from hossam import load_data
from pandas import DataFrame, merge, get_dummies
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [2]:
customers_df = load_data('online_store_customers')
customers_df

[data] https://data.hossam.kr/data/lab05/online_store_customers.xlsx
[desc] 한 온라인 스토어의 마케팅 팀이 연말 특별 프로모션을 기획에 필요한 고객의 기본 인구 통계 정보(메타데이터 없음)
[!] Cannot read metadata


,name,gender,age
user_id,,,
1,Alice,F,25
2,Bob,M,30
3,Charlie,M,35
4,David,M,42
5,Eve,F,28
6,Frank,M,21
7,Grace,F,33
8,Henry,M,45
9,Ivy,F,29


In [3]:
purchases_df = load_data('online_store_purchases')
purchases_df

[data] https://data.hossam.kr/data/lab05/online_store_purchases.xlsx
[desc] 한 온라인 스토어의 마케팅 팀이 연말 특별 프로모션을 기획에 필요한 고객의 구매 관련 정보(메타데이터 없음)
[!] Cannot read metadata


,user_id,product,size,color,price
purchase_id,,,,,
101,1,T-shirt,M,White,15000
102,2,Pants,L,Black,35000
103,1,Skirt,S,Red,25000
104,3,T-shirt,L,Blue,17000
105,5,Jacket,M,Black,55000
106,4,Pants,M,Khaki,33000
107,6,T-shirt,S,White,16000
108,7,Jacket,NaN,Red,58000
109,8,Pants,L,Black,38000


1.데이터 병합

In [4]:
merged_df = merge(customers_df, purchases_df, left_index=True, right_on='user_id')
merged_df

,name,gender,age,user_id,product,size,color,price
purchase_id,,,,,,,,
101,Alice,F,25,1,T-shirt,M,White,15000
103,Alice,F,25,1,Skirt,S,Red,25000
115,Alice,F,25,1,Pants,S,Blue,31000
102,Bob,M,30,2,Pants,L,Black,35000
113,Bob,M,30,2,Jacket,L,Navy,62000
104,Charlie,M,35,3,T-shirt,L,Blue,17000
106,David,M,42,4,Pants,M,Khaki,33000
105,Eve,F,28,5,Jacket,M,Black,55000
112,Eve,F,28,5,Pants,S,Khaki,32000


In [5]:
imr = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imr_value = imr.fit_transform(merged_df[['size']])
imr_value

array([['M'],
       ['S'],
       ['S'],
       ['L'],
       ['L'],
       ['L'],
       ['M'],
       ['M'],
       ['S'],
       ['S'],
       ['S'],
       ['M'],
       ['L'],
       ['S'],
       ['XL']], dtype=object)

In [6]:
imr_df = DataFrame(imr_value, index = merged_df.index, columns=['size'])
imr_df

,size
purchase_id,
101,M
103,S
115,S
102,L
113,L
104,L
106,M
105,M
112,S


In [8]:
merged_copy_df = merged_df.copy()
merged_copy_df['size'] = imr_df['size']
merged_copy_df

,name,gender,age,user_id,product,size,color,price
purchase_id,,,,,,,,
101,Alice,F,25,1,T-shirt,M,White,15000
103,Alice,F,25,1,Skirt,S,Red,25000
115,Alice,F,25,1,Pants,S,Blue,31000
102,Bob,M,30,2,Pants,L,Black,35000
113,Bob,M,30,2,Jacket,L,Navy,62000
104,Charlie,M,35,3,T-shirt,L,Blue,17000
106,David,M,42,4,Pants,M,Khaki,33000
105,Eve,F,28,5,Jacket,M,Black,55000
112,Eve,F,28,5,Pants,S,Khaki,32000


In [11]:
dm_df1 = get_dummies(merged_copy_df, columns=['gender'], drop_first=True, dtype='int')
dm_df1.head()

,name,age,user_id,product,size,color,price,gender_M
purchase_id,,,,,,,,
101,Alice,25,1,T-shirt,M,White,15000,0
103,Alice,25,1,Skirt,S,Red,25000,0
115,Alice,25,1,Pants,S,Blue,31000,0
102,Bob,30,2,Pants,L,Black,35000,1
113,Bob,30,2,Jacket,L,Navy,62000,1


In [13]:
encoder = OneHotEncoder(sparse_output=False, drop="first")
result = encoder.fit_transform(dm_df1[['color']])
result

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.]])

In [15]:
new_cols = encoder.get_feature_names_out(['color'])
one_hot_df = DataFrame(result, columns=new_cols, index=dm_df1.index)
one_hot_df.head()

,color_Blue,color_Khaki,color_Navy,color_Red,color_White
purchase_id,,,,,
101,0.0,0.0,0.0,0.0,1.0
103,0.0,0.0,0.0,1.0,0.0
115,1.0,0.0,0.0,0.0,0.0
102,0.0,0.0,0.0,0.0,0.0
113,0.0,0.0,1.0,0.0,0.0


In [16]:
dm_df2 = merge(dm_df1, one_hot_df, left_index=True, right_index=True)
dm_df2.drop(columns='color', inplace=True)
dm_df2.head()

,name,age,user_id,product,size,price,gender_M,color_Blue,color_Khaki,color_Navy,color_Red,color_White
purchase_id,,,,,,,,,,,,
101,Alice,25,1,T-shirt,M,15000,0,0.0,0.0,0.0,0.0,1.0
103,Alice,25,1,Skirt,S,25000,0,0.0,0.0,0.0,1.0,0.0
115,Alice,25,1,Pants,S,31000,0,1.0,0.0,0.0,0.0,0.0
102,Bob,30,2,Pants,L,35000,1,0.0,0.0,0.0,0.0,0.0
113,Bob,30,2,Jacket,L,62000,1,0.0,0.0,1.0,0.0,0.0
